In [2]:
from util import data_generate, train, cnnlstm, cnn3d, DPtrain

import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from opacus.layers import DPLSTM
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager

from opacus.validators import ModuleValidator

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load data
batch_size = 512
path_train = './train_split'
path_test = './test_split'


train_dataset = data_generate.VideoFrameDataset(path_train, 196)
test_dataset = data_generate.VideoFrameDataset(path_test, 196)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function, and optimizer
num_classes = 10  # Set this to the number of classes in your dataset
num_epochs = 1

In [6]:
model = cnn3d.cnn3d(num_classes=10).to('cuda')
criterion = nn.CrossEntropyLoss(reduction='none')  # Per-sample loss

errors = ModuleValidator.validate(model, strict=False)
errors[-5:]

model = ModuleValidator.fix(model)

EPSILON = 1
MAX_GRAD_NORM = 1.2
DELTA = 1e-5

num_epochs = 1
EPOCHS = num_epochs

privacy_engine = PrivacyEngine()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    poisson_sampling = False,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

epoch_results_DP3dcnn = DPtrain.DPcnn3d(model, train_loader, test_loader, criterion, optimizer, num_epochs,EPSILON)

Epoch [1/1], Train Loss: 1.9611, Train Accuracy: 0.3466, Val Loss: 2.5682, Val Accuracy: 0.3109
Interval Accuracies: {0: 0.2667022982362373, 1: 0.26082308925708175, 2: 0.2661678246926777, 3: 0.3180117584179583, 4: 0.3254943880277926, 5: 0.3185462319615179, 6: 0.3185462319615179, 7: 0.32923570283270975, 8: 0.3436664885088188, 9: 0.361838588989845}
